In [1]:
import torch
import torchvision 
import torchaudio

import os

# yesno

In [254]:
# yesno
# filename = "waves_yesno.tar.gz"
# url = "http://www.openslr.org/resources/1/" + filename

url = "http://www.openslr.org/resources/1/waves_yesno.tar.gz"
filename = os.path.basename(url)
root = "/Users/vincentqb/yesnotest/"

inpath = root + filename
dset_abs_path = root + "waves_yesno/"

In [3]:
torchvision.datasets.utils.download_url(url, root)

# torchvision.datasets.utils.extract_archive(inpath, root)  # FIXME no extraction with root
torchvision.datasets.utils.extract_archive(inpath)

Using downloaded and verified file: /Users/vincentqb/yesnotest/waves_yesno.tar.gz


In [255]:
def iterate_yesno(dset_abs_path):
    for f in os.listdir(dset_abs_path):
        if ".wav" not in f:
            continue
    
        full_path = os.path.join(dset_abs_path, f)
        waveform, sample_rate = torchaudio.load(full_path)
        label = os.path.basename(f).split(".", 1)[0].split("_")
        l = sig.size(1)
    
        yield label, waveform, sample_rate, l

g = iterate_yesno(dset_abs_path)

In [256]:
len(list(g))

60

# vctk

In [257]:
filename1 = "DS_10283_2651.zip"
filename2 = "VCTK-Corpus.zip"
url = "http://datashare.is.ed.ac.uk/download/" + filename
root = "/Users/vincentqb/vctktest/"

inpath1 = root + filename1
inpath2 = root + filename2
dset_abs_path = root + "VCTK-Corpus/"

In [258]:
# torchvision.datasets.utils.download_url(url, root)

# torchvision.datasets.utils.extract_archive(inpath, root)  # FIXME no extraction with root
torchvision.datasets.utils.extract_archive(inpath1)
torchvision.datasets.utils.extract_archive(inpath2)

In [259]:
def iterate_vctk(root):
    folder_txt = os.path.join(root, "txt/")
    folder_wav = os.path.join(root, "wav48/")
    
    for dp, _, fn in os.walk(folder_txt):
        for f in fn:
            if ".txt" not in f:
                continue
            
            file_txt = os.path.join(dp, f)
            with open(file_txt) as file_txt:
                content = file_txt.readlines()[0]

            
            base = os.path.basename(f).split(".", 1)[0]
            folder = base.split("_")[0]
            file_audio = os.path.join(folder_wav, folder, base) + ".wav"
            waveform, sample_rate = torchaudio.load(file_audio)
            
            yield base, content, waveform, sample_rate

g = iterate_vctk(dset_abs_path)

In [260]:
next(g)

('p304_225',
 'I see no reason for them to be.',
 tensor([[-0.0038, -0.0046, -0.0047,  ..., -0.0019, -0.0019, -0.0017]]),
 48000)

# librispeech

In [261]:
# http://www.openslr.org/12
    
# http://www.openslr.org/resources/12/dev-clean.tar.gz
# http://www.openslr.org/resources/12/test-clean.tar.gz
# http://www.openslr.org/resources/12/test-clean.tar.gz
# http://www.openslr.org/resources/12/test-other.tar.gz
# http://www.openslr.org/resources/12/train-clean-100.tar.gz
# http://www.openslr.org/resources/12/train-clean-360.tar.gz
# http://www.openslr.org/resources/12/train-other-500.tar.gz

url = "http://www.openslr.org/resources/12/dev-clean.tar.gz"
root = "/Users/vincentqb/librispeechtest/"

filename = os.path.basename(url)
inpath = root + filename

base = filename.split(".")[0]
dset_abs_path = root + "LibriSpeech/" + base + "/"

In [163]:
# torchvision.datasets.utils.download_url(url, root)
# torchvision.datasets.utils.extract_archive(inpath)

In [262]:
def iterate_librispeech(dset_abs_path):
    for dp, dn, fn in os.walk(dset_abs_path):
        for f in fn:
            if ".txt" not in f:
                continue
        
            fp = os.path.join(dp, f)
            for line in open(fp):
                fileid, content = line.strip().split(" ", 1)
                flac = os.path.join(dp, fileid) + ".flac"
                waveform, sample_rate = torchaudio.load(flac)
                yield fileid, waveform, sample_rate, content

g = iterate_librispeech(dset_abs_path)

In [263]:
next(g)

('2412-153954-0000',
 tensor([[-3.0518e-05,  0.0000e+00,  3.0518e-05,  ..., -1.2207e-04,
          -1.2207e-04, -9.1553e-05]]),
 16000,
 'SHORTLY AFTER PASSING ONE OF THESE CHAPELS WE CAME SUDDENLY UPON A VILLAGE WHICH STARTED UP OUT OF THE MIST AND I WAS ALARMED LEST I SHOULD BE MADE AN OBJECT OF CURIOSITY OR DISLIKE')

# CommonVoice

In [265]:
web = "https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-3/"

languages = {
    "tatar": "tt",
    "english": "en",
    "german": "de",
    "french": "fr",
    "welsh": "cy",
    "breton": "br",
    "chuvash": "cv",
    "turkish": "tr",
    "kyrgyz": "ky",
    "irish": "ga-IE",
    "kabyle": "kab",
    "catalan": "ca",
    "taiwanese": "zh-TW",
    "slovenian": "sl",
    "italian": "it",
    "dutch": "nl",
    "hakha chin": "cnh",
    "esperanto": "eo",
    "estonian": "et",
    "persian": "fa",
    "basque": "eu",
    "spanish": "es",
    "chinese": "zh-CN",
    "mongolian": "mn",
    "sakha": "sah",
    "dhivehi": "dv",
    "kinyarwanda": "rw",
    "swedish": "sv-SE",
    "russian": "ru",
}

url = web + languages["tatar"] + ".tar.gz"
root = "/Users/vincentqb/commonvoicetest/"

filename = os.path.basename(url)
inpath = root + filename

base = filename.split(".")[0]
dset_abs_path = root
tsv = "train.tsv"

In [210]:
# torchvision.datasets.utils.download_url(url, root)
# torchvision.datasets.utils.extract_archive(inpath)

In [273]:
def iterate_commonvoice(dset_abs_path, tsv):
    f = os.path.join(dset_abs_path, tsv)
    with open(f) as f:
        next(f)  # skip header line
        for line in f:
            line = line.strip().split("\t")
            filename = os.path.join(dset_abs_path, "clips", line[1])
            waveform, sample_rate = torchaudio.load(filename)

            # client_id, path, sentence, up_votes, down_votes, age, gender, accent, waveform, sample_rate
            line.extend((waveform, sample_rate))
            yield line

g = iterate_commonvoice(dset_abs_path, tsv)

In [274]:
next(g)

['11d5e99f7bd5b4f8492a06bb1ec22aa9110bba6ea9918f2a9adec05d686304d568ab7063daf8915d3fccfb4dd44b81646bd13a33ca130ac4014560bba4c2db0b',
 'common_voice_tt_17343438.mp3',
 'Баш өсте, хан хәзрәтләре.',
 '2',
 '0',
 'thirties',
 'male',
 tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -5.3644e-06,
          -1.7546e-06,  2.0936e-06]]),
 48000]